# Graficos

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import csv

In [ ]:
dataframe = pd.read_csv('output.csv', sep=';')

tiempos_promedio = []
valor_inicial = 100

for i in range(16):
    tiempos_promedio.append(float(dataframe.iloc[valor_inicial, 0])/100)
    valor_inicial += 102

etiquetas_x = [f'2^{i}' for i in range(10, 26)]

plt.figure(figsize=(10, 6))
plt.plot(etiquetas_x, tiempos_promedio, marker='o', linestyle='-')
plt.xlabel('Tamaño de entrada')
plt.ylabel('Tiempos promedio (µs)')
plt.title('Tiempos promedio en función del tamaño de entrada')
plt.xticks(rotation=45)
plt.tight_layout()

plt.savefig('resultados/grafico_tiempos_promedio.pdf')
plt.show()

In [ ]:
suma_ios = []
valor_columna = 100

for i in range(16):
    suma_ios.append(int(dataframe.iloc[valor_columna, 1]))
    valor_columna += 102

etiquetas_x = [f'2^{i}' for i in range(10, 26)]

plt.figure(figsize=(10, 6))
plt.plot(etiquetas_x, suma_ios, marker='o', linestyle='-')
plt.xlabel('Tamaño de entrada')
plt.ylabel('I/Os (suma de todas las consultas)')
plt.title('I/Os')
plt.xticks(rotation=45)
plt.tight_layout()

plt.savefig('resultados/grafico_ios.pdf')
plt.show()